In [1]:
%matplotlib qt
import os
import os.path as op
from pathlib import Path
import pickle
import numpy as np
from mne.datasets import fetch_fsaverage
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import mne
from mne_icalabel import label_components
from tqdm.contrib.itertools import product
from mne.stats import permutation_t_test
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs
from mne_connectivity import spectral_connectivity_epochs
from learn_graph import log_degree_barrier
from scipy.stats import ttest_ind
import seaborn as sns

/Users/payamsadeghishabestari/virtual_envs/.venv/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
## initialize parameters
sfreq = 250.0
(l_freq, h_freq, no_freq) = (0.1, 100.0, 50.0)
montage = mne.channels.make_standard_montage("easycap-M1")
verbose = False
fs_dir = fetch_fsaverage(verbose=verbose)
src_fname = op.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = op.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
bands_dict = {"delta": [0.5, 4], "theta": [4, 8], "alpha": [8, 13],
            "beta": [13, 30], "gamma": [30, 80]}

snr = 1.0  
lambda2 = 1.0 / snr**2
brain_labels = mne.read_labels_from_annot(subject="fsaverage", parc="aparc", verbose=verbose)[:-1]
bl_names = [bl.name for bl in brain_labels]
power_keys = ["subject_ID", "hemisphere", "protocol", "run", "frequency_band", "brain_label", "power"]

## initialize parameters
compute_power = True
compute_conn = True
compute_graph = True
create_report = True

## loop over subjects
folder_name = Path.cwd().parent / "data" / "EEG_data"
for file in tqdm(sorted(os.listdir(folder_name))[300:]):
    if file.endswith(".set"):
        fname = folder_name / file
        subject_id = file[:5]
        if file[8:9] == "R": hemisphere = "right"
        if file[8:9] == "L": hemisphere = "left"
        if file[-7:-4] == "pre": run = "pre"
        if file[-8:-4] == "post": run = "post"
        if "_0.1Hz" in file: protocol = "0.1Hz"
        if "_1Hz" in file: protocol = "1Hz"
        if "_10Hz" in file: protocol = "10Hz"
        if "_20Hz" in file: protocol = "20Hz"

        my_dict = {}
        power_dict = {key: [] for key in power_keys}
        con_dict = {}
        graphs_dict = {}

        ## read raw eeg files
        raw = mne.io.read_raw_eeglab(input_fname=fname, preload=True, verbose=verbose)
        raw.annotations.delete(range(len(raw.annotations)))

        ## set montage (FPz -> Fpz)
        raw.set_montage(montage=montage, match_case=False, on_missing="raise")

        ## resample and filter, re-referencing
        raw.resample(sfreq=sfreq, verbose=verbose)
        raw.filter(l_freq=l_freq, h_freq=h_freq, verbose=verbose)
        raw.notch_filter(freqs=no_freq, verbose=verbose) 
        raw.set_eeg_reference("average", projection=False, verbose=verbose)

        ## ICA
        ica = mne.preprocessing.ICA(n_components=0.95, max_iter=800, method='infomax',
                                    fit_params=dict(extended=True))
        try:
            ica.fit(raw, verbose=verbose)
        except:
            ica = mne.preprocessing.ICA(n_components=5, max_iter=800, method='infomax',
                                    fit_params=dict(extended=True))
            ica.fit(raw, verbose=verbose)
        
        ic_dict = label_components(raw, ica, method="iclabel")
        ic_labels = ic_dict["labels"]
        ic_probs = ic_dict["y_pred_proba"]
        eog_indices = []
        for idx, label in enumerate(ic_labels):
            if label == "eye blink" and ic_probs[idx] > 0.70:
                eog_indices.append(idx)

        if len(eog_indices) > 0:
            eog_components = ica.plot_properties(raw, picks=eog_indices, verbose=verbose, show=False)
            for fig_idx, fig in enumerate(eog_components):
                pred_value = int(ic_probs[eog_indices[fig_idx]] * 1e2)
                fig.axes[0].set_title(f"pred score: 0.{pred_value}")
        
        ica.apply(raw, exclude=eog_indices, verbose=verbose)
        
        ## sensor analysis
        epochs = mne.make_fixed_length_epochs(raw, duration=5, verbose=verbose)
        for frange, freqs in bands_dict.items():
            psd_vals = epochs.compute_psd(fmin=freqs[0], fmax=freqs[1],
                                        verbose=verbose).get_data().mean(axis=2)
            
            my_dict[f"{frange}"] = psd_vals
        
        fname_save = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "sensor" / f"{subject_id}_{protocol}_{hemisphere}_{run}.pkl"
        with open(fname_save, "wb") as file_pkl:
            pickle.dump(my_dict, file_pkl)

        ## source power
        info = raw.info
        fwd = mne.make_forward_solution(info, trans="fsaverage", src=src_fname, bem=bem, eeg=True,
                                        meg=False, verbose=verbose)
        noise_cov = mne.make_ad_hoc_cov(info, std=None, verbose=verbose)
        inverse_operator = make_inverse_operator(info, fwd, noise_cov, verbose=verbose)
        src = inverse_operator["src"]
        raw.set_eeg_reference("average", projection=True, verbose=verbose)
        
        for band, (freq_l, freq_h) in bands_dict.items(): 
            print(f"working on {file} at frequency range {band}.")
            raw_filt = raw.copy().filter(freq_l, freq_h, verbose=verbose)
            epochs = mne.make_fixed_length_epochs(raw_filt, duration=5.0, preload=True, verbose=verbose)
            
            ## compute power in labels (try to not fill memory)
            if compute_power:
                stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method="dSPM",
                                            label=None, return_generator=True, verbose=verbose)
                label_ts = mne.extract_label_time_course(stcs, brain_labels, src, mode="mean",
                                                        return_generator=False, verbose=verbose)
                powers = np.array(label_ts).mean(axis=(0, 2))
                del stcs, label_ts
                
                ## fill in the dataframe
                for label, power in zip(brain_labels, powers):
                    power_dict["subject_ID"].append(subject_id)
                    power_dict["hemisphere"].append(hemisphere)
                    power_dict["protocol"].append(protocol)
                    power_dict["run"].append(run)
                    power_dict["frequency_band"].append(band)
                    power_dict["brain_label"].append(label.name)
                    power_dict["power"].append(power)

            ## compute connectivity between labels (wPLI, Coh)
            if compute_conn:
                stcs = apply_inverse_epochs(epochs, inverse_operator, lambda2, method="dSPM",
                                            label=None, return_generator=True, verbose=verbose)
                label_ts = mne.extract_label_time_course(stcs, brain_labels, src, mode="mean",
                                                        return_generator=False, verbose=verbose)
                conns = spectral_connectivity_epochs(data=label_ts, names=bl_names,
                                                    method=["wpli", "coh"], sfreq=sfreq,
                                                    fmin=freq_l, fmax=freq_h, faverage=True,
                                                    verbose=verbose)
                del stcs
                con_data = np.array([con.get_data(output="dense")[:, :, 0] for con in conns])
                con_dict[band] = con_data

            ## learn graph on FC data
            if compute_graph:
                label_ts_reshaped = np.array(label_ts).transpose(1, 0, 2)
                X = label_ts_reshaped.reshape(label_ts_reshaped.shape[0], -1)
                graph = log_degree_barrier(X=X, dist_type='sqeuclidean', alpha=1,
                                                beta=1, step=0.5, w0=None, maxit=10000, rtol=1e-16,
                                                retall=False, verbosity='NONE')
                graphs_dict[band] = graph

        ## fill and save the report
        if compute_power: 
            fname_save = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "powers" / f"{file[:-4]}.csv"
            df_power = pd.DataFrame(power_dict)
            df_power.to_csv(fname_save)

        if compute_conn: 
            fname_save = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "conns" / f"{file[:-4]}.pkl"
            with open(fname_save, "wb") as file_pkl:
                pickle.dump(con_dict, file_pkl)

        if compute_graph:
            fname_save = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "graphs" / f"{file[:-4]}.pkl"
            with open(fname_save, "wb") as file_pkl:
                pickle.dump(graphs_dict, file_pkl)
        
        if create_report:
            report = mne.Report(title=f"report_subject_{subject_id}", verbose=verbose)
            report.add_raw(raw=raw, title="recording info", butterfly=False, psd=False) 
            if len(eog_indices) > 0:
                report.add_figure(fig=eog_components, title="EOG components", image_format="PNG")
            fname_report = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "reports" / f"{file[:-4]}.html"
            report.save(fname=fname_report, open_browser=False, overwrite=True, verbose=verbose)

Check the difference in sensor space (epoch wise)

In [ ]:
raw = mne.io.read_raw_eeglab(input_fname="/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_2_L_0.1Hz_post.set",
                            preload=True, verbose=False)
montage = mne.channels.make_standard_montage("easycap-M1")
raw.annotations.delete(range(len(raw.annotations)))
raw.set_montage(montage=montage, match_case=False, on_missing="raise")
info = raw.info

folder_name = Path.cwd().parent / "data" / "dataframes" / "visit_2" / "sensor"
subject_ids = []
for file in sorted(os.listdir(folder_name)):
    if file.endswith(".pkl"):
        subject_ids.append(file[:5])
subject_ids = list(np.unique(np.array(subject_ids)))    
subjects_to_drop = ["3dx2e", "6sjul", "musky", "8kmc7", "cmh15", "uvxfg", "2fjeu", "dws0m", "gigcm"]

protocols = ["0.1Hz", "1Hz", "10Hz", "20Hz"][:1]
hemis = ["left", "right"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
combs = product(protocols, hemis, franges)

for protocol, hemi, frange in combs:
    x1, x2 = [], []
    for subject_id in subject_ids:
        if subject_id not in subjects_to_drop:
            fname_pre = folder_name / f"{subject_id}_{protocol}_{hemi}_pre.pkl"
            fname_post = folder_name / f"{subject_id}_{protocol}_{hemi}_post.pkl"

            with open(fname_pre, "rb") as fpre:
                dict_pre = pickle.load(fpre)
            with open(fname_post, "rb") as fpost:
                dict_post = pickle.load(fpost)

            x1.append(dict_pre[frange])
            x2.append(dict_post[frange])

    x1 = np.concatenate(x1, axis=0)
    x2 = np.concatenate(x2, axis=0)
    n_epochs = min(len(x1), len(x2))

    X = x2[:n_epochs] - x1[:n_epochs]
    T0, p_values, H0 = permutation_t_test(X=X, n_permutations=10000)
    
    evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis], info, tmin=0.0)
    mask = p_values[:, np.newaxis] <= 0.05
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    evoked.plot_topomap(
        ch_type="eeg",
        times=[0],
        scalings=1,
        colorbar=False,
        time_format=None,
        cmap=None,
        vlim=(0, np.max),
        units="log10(p)",
        cbar_fmt="%0.1f",
        mask=mask,
        size=3,
        show_names=False,
        time_unit="s",
        axes=ax
    )
    ax.set_title(f"{protocol}_{hemi}_{frange}")

Check the difference in sensor space (subject wise)

In [3]:
## define responders and non responders
subjects_to_drop_dict = {}
prots = ["1 Hz", "10 Hz", "20 Hz"]
hemis = ["left", "right"]
all_combs = product(prots, hemis)
for protocol, hemi in all_combs:
    fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data.xlsx"
    df_tl = pd.read_excel(fname)
    df_tl["protcol"] = df_tl["protcol"].replace("1Hz", "1 Hz")
    df_tl_sub = df_tl.query(f'protcol == "{protocol}" & hemisphere == "{hemi}"')
    df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
    df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
    df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
    df_tl_sub = df_tl_sub.rename(columns={'ID': 'subject_ID'})
    df_tl_diff = df_tl_sub[["subject_ID", "diff_TL"]]
    df_tl_diff = df_tl_diff.query('diff_TL <= 0')
    subjects_to_drop_dict[f"{hemi}_{protocol}"] = list(df_tl_diff["subject_ID"].values)

  0%|          | 0/6 [00:00<?, ?it/s]

/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1542/3124130704.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1542/3124130704.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1542/3124130704.py:13: SettingWithCopyWarning: 
A value is trying to be s

In [5]:
raw = mne.io.read_raw_eeglab(input_fname="/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_2_L_0.1Hz_post.set",
                            preload=True, verbose=False)
montage = mne.channels.make_standard_montage("easycap-M1")
raw.annotations.delete(range(len(raw.annotations)))
raw.set_montage(montage=montage, match_case=False, on_missing="raise")
info = raw.info

folder_name = Path.cwd().parent / "data" / "dataframes" / "sensor"
subject_ids = []
for file in sorted(os.listdir(folder_name)):
    if file.endswith(".pkl"):
        subject_ids.append(file[:5])
subject_ids = list(np.unique(np.array(subject_ids)))
# subjects_to_drop = ["3dx2e", "6sjul", "musky", "8kmc7", "cmh15", "uvxfg", "2fjeu", "dws0m", "gigcm"]

protocols = ["0.1Hz", "1Hz", "10Hz", "20Hz"][3:4]
hemis = ["left", "right"]
franges = ["delta", "theta", "alpha", "beta", "gamma"]
combs = product(protocols, hemis, franges)

for protocol, hemi, frange in combs:
    subjects_to_drop = subjects_to_drop_dict[f"{hemi}_{protocol[:-2]} Hz"] 
    x1, x2 = [], []
    for subject_id in subject_ids:
        if subject_id not in subjects_to_drop:
            fname_pre = folder_name / f"{subject_id}_{protocol}_{hemi}_pre.pkl"
            fname_post = folder_name / f"{subject_id}_{protocol}_{hemi}_post.pkl"

            with open(fname_pre, "rb") as fpre:
                dict_pre = pickle.load(fpre)
            with open(fname_post, "rb") as fpost:
                dict_post = pickle.load(fpost)

            x1.append(dict_pre[frange].mean(axis=0))
            x2.append(dict_post[frange].mean(axis=0))

    X = np.array(x2) - np.array(x1)
    # t_values, p_values, H0 = permutation_t_test(X=X, n_permutations=10000)
    T0, p_values = ttest_ind(np.array(x1), np.array(x2))
    
    evoked = mne.EvokedArray(-np.log10(p_values)[:, np.newaxis], info, tmin=0.0)
    mask = p_values[:, np.newaxis] <= 0.05
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    evoked.plot_topomap(
        ch_type="eeg",
        times=[0],
        scalings=1,
        colorbar=False,
        time_format=None,
        cmap=None,
        vlim=(0, np.max),
        units="log10(t)",
        cbar_fmt="%0.1f",
        mask=mask,
        size=3,
        show_names=False,
        time_unit="s",
        axes=ax
    )
    ax.set_title(f"{protocol}_{hemi}_{frange}")

/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1542/1030094895.py:1: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(input_fname="/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_2_L_0.1Hz_post.set",
/var/folders/20/hsy69tx529ndn3rkv5gzcf0c0000gn/T/ipykernel_1542/1030094895.py:1: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(input_fname="/Users/payamsadeghishabestari/codes/regTMS/data/EEG_data/2bbyx_2_L_0.1Hz_post.set",


  0%|          | 0/10 [00:00<?, ?it/s]

Correlation of gamma change and Tinnitus change

In [ ]:
fname = "/Users/payamsadeghishabestari/codes/regTMS/data/behavioral_data/TL_data.xlsx"
subjects_to_drop = ["3dx2e", "6sjul", "musky", "8kmc7", "cmh15", "uvxfg", "2fjeu", "dws0m", "gigcm"]
df_tl = pd.read_excel(fname)
df_tl_sub = df_tl.query('ID != @subjects_to_drop & protcol == "20 Hz" & hemisphere == "right"')
df_tl_sub['pre_avg'] = df_tl.filter(like='pre').mean(axis=1)
df_tl_sub['post_avg'] = df_tl.filter(like='post').mean(axis=1)
df_tl_sub["diff_TL"] = df_tl_sub['pre_avg'] - df_tl_sub['post_avg']
df_tl_sub = df_tl_sub.rename(columns={'ID': 'subject_ID'})
df_tl_diff = df_tl_sub[["subject_ID", "diff_TL"]]

In [ ]:
## load power in labels and add whole brain activation
dfs_list = []
folder_name = Path.cwd().parent / "data" / "dataframes" / "powers"
for file in tqdm(sorted(os.listdir(folder_name))):
    if file.endswith(".csv"):
        fname = folder_name / file
        df = pd.read_csv(fname, index_col=[0])
        
        ## add whole brain power
        df1s_list = []
        for frange in ["delta", "theta", "alpha", "beta", "gamma"]:
            for hemi in ["lh", "rh"]:
                df1 = df[df["frequency_band"] == frange]
                df1 = df1[df1['brain_label'].str.contains(f'-{hemi}', case=False, na=False)]
                power_hemi = df1["power"].sum()
                keys = list(df.columns)
                values = [df1[key].unique()[0] for key in keys[:-2]]
                values.append([f"whole-{hemi}"])
                values.append([power_hemi])
                new_row = dict(zip(keys, values))
                df_new_row = pd.DataFrame(new_row)
                df1 = pd.concat([df1, df_new_row])
                df1s_list.append(df1)
        df_updated = pd.concat(df1s_list)
        dfs_list.append(df_updated)
        
dfs = pd.concat(dfs_list)
df_eeg = dfs.reset_index().drop(columns=["index", "Unnamed: 0"])

In [ ]:
dfs_list = []
bls = ["frontalpole-rh", "lateralorbitofrontal-rh", "medialorbitofrontal-lh", "medialorbitofrontal-rh", "rostralanteriorcingulate-rh"]
for bl in bls:
    df_sub = df_eeg.query(f'subject_ID != @subjects_to_drop & hemisphere == "right" & protocol == "20 Hz" & frequency_band == "gamma" & brain_label == "{bl}"')
    df_pivot = df_sub.pivot(index='subject_ID', columns='run', values='power')
    df_pivot['power_difference'] = df_pivot['post'] - df_pivot['pre']
    df_pivot = df_pivot.reset_index()
    df_eeg_diff = df_pivot[["subject_ID", "power_difference"]]
    df_corr = pd.merge(df_eeg_diff, df_tl_diff, on='subject_ID', how='inner')
    df_corr["brain_label"] = [bl] * len(df_corr)
    dfs_list.append(df_corr)
df_corr = pd.concat(dfs_list)
pal = sns.cubehelix_palette(2, rot=-.25, light=.7, as_cmap=False)
sns.lmplot(data=df_corr, x="diff_TL", y="power_difference", col="brain_label", palette=pal, scatter_kws={"s": 30, "alpha": 1})

In [ ]:
df_corr